In [233]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from funpymodeling.exploratory import freq_tbl
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [234]:
df = pd.read_csv('Roma.csv')

In [235]:
df= df.fillna(method="bfill")
df= df.fillna(method="ffill")
df = df.bfill()
df= df.ffill()

C:\Users\diego\AppData\Local\Temp\ipykernel_2532\1730909967.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method="bfill")
C:\Users\diego\AppData\Local\Temp\ipykernel_2532\1730909967.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method="ffill")


**Transformando variables a tipo dicotómicas**

In [236]:

#bathrooms_text
df["bathroomsd"] = df["bathrooms_text"].astype(str).apply(lambda x: 1 if "2" in x or "3" in x or "4" in x else 0)
#room_type Entire home
df["E_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Entire home/apt" else 0)
#room_type Private home
df["P_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Private room" else 0)
#room_type Hotel home
df["H_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Hotel room" else 0)
#instant_bookabled
df["instant_bookabled"] = df["instant_bookable"].apply(lambda x: 1 if x == "t" else 0)
#Price
df["price"] = df["price"].replace('[\$,]', '', regex=True).astype(float)
Mprice = df["price"].median()
df["D_price"] = df["price"].apply(lambda x: 1 if x >= Mprice else 0)
#host_total_listings_count
Mlistings = df["host_total_listings_count"].median()
df["D_host_listings"] = df["host_total_listings_count"].apply(lambda x: 1 if x >= Mlistings else 0)
#host_total_listings_count
Maccuracy = df["host_total_listings_count"].median()
df["D_accuracy"] = df["review_scores_accuracy"].apply(lambda x: 1 if x >= Maccuracy else 0)
#availability_30
Mavailability = df["availability_30"].median()
df["D_availability"] = df["availability_30"].apply(lambda x: 1 if x >= Mavailability else 0)
#availability_90
Mavailability90 = df["availability_90"].median()
df["D_availability90"] = df["availability_90"].apply(lambda x: 1 if x >= Mavailability90 else 0)
#number_of_reviews
Mreviews = df["number_of_reviews"].median()
df["D_reviews"] = df["number_of_reviews"].apply(lambda x: 1 if x >= Mreviews else 0)
#review_scores_rating
Mrating = df["review_scores_rating"].median()
df["D_rating"] = df["review_scores_rating"].apply(lambda x: 1 if x >= Mrating else 0)
#review_scores_rating
Maccommodates = df["accommodates"].median()
df["D_accommodates"] = df["accommodates"].apply(lambda x: 1 if x >= Maccommodates else 0)



<>:12: SyntaxWarning: invalid escape sequence '\$'
<>:12: SyntaxWarning: invalid escape sequence '\$'
C:\Users\diego\AppData\Local\Temp\ipykernel_2532\2275433636.py:12: SyntaxWarning: invalid escape sequence '\$'
  df["price"] = df["price"].replace('[\$,]', '', regex=True).astype(float)


In [237]:
df.head(5)

Unnamed: 0       id                                               name  \
0           0   2737.0                   Elif's room in cozy, clean flat.   
1           1   3079.0          Cozy apartment  (2-4)with Colisseum  view   
2           3  12398.0                Casa Donatello - Home far from Home   
3           4  19965.0   S.Peter's Basilica 5 MIN WALK two large bedrooms   
4           5  50333.0  Bright&Cosy Penthouse Overlooking St. Peter's ...   

                                         description  \
0  10 min by bus you can get to Piazza Venezia or...   
1  With the view of the Colisseum from the front ...   
2  Casa Donatello is a newly renovated two-bedroo...   
3  Bright and quiet two bedrooms apartment 3rd fl...   
4  Very bright and comfortable, impressive views ...   

                               neighborhood_overview  host_id  \
0  It used to be an industrial area until late 80...     3047   
1  Monti neighborhood is one of the best areas in...     3504   
2  You are at 15 minutes walking distance from hi...    11756   
3  Prati is a famous neighbourhood (rione of Rome...    75450   
4  There are plenty of shops nearby, small restau...   230736   

         host_name  host_since host_location  host_response_time  ...  \
0             Elif  18/09/2008   Rome, Italy      within an hour  ...   
1            Laura  08/10/2008   Rome, Italy  within a few hours  ...   
2              Gea  30/03/2009   Rome, Italy      within an hour  ...   
3             Sasa  30/01/2010   Rome, Italy      within an hour  ...   
4  Sita Alessandra  10/09/2010   Rome, Italy      within an hour  ...   

  H_room_typed instant_bookabled D_price D_host_listings  D_accuracy  \
0            0                 1       0               1           0   
1            0                 1       1               1           0   
2            0                 0       1               1           0   
3            0                 0       1               1           0   
4            0                 0       1               0           0   

   D_availability D_availability90 D_reviews  D_rating  D_accommodates  
0               1                1         0         1               0  
1               0                0         0         0               1  
2               1                1         1         1               1  
3               1                1         1         0               1  
4               1                1         1         1               0  

[5 rows x 79 columns]

**CASO 1: accommodates, bathrooms, room_type**

In [238]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_accommodates', 'bathroomsd', 'E_room_typed']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,

In [239]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[723 403]
 [391 702]]


In [240]:
#Caulculo de la precisuin del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6352941176470588


In [241]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6421811626858945


In [242]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6422689844464776


**CASO 2:** 
**price, host_listings, accuracy**

In [243]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_price', 'D_host_listings', 'D_accuracy']]
Var_Dep = df['D_availability']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,

In [244]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[612 328]
 [490 789]]


In [245]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.7063563115487914


In [246]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6313654799459216


In [247]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6168881939014855


**CASO 3:** 
**availability, number_of_reviews, review_scores_rating**

In [248]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_availability', 'D_reviews', 'D_rating']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,

In [249]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[610 506]
 [351 752]]


In [250]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5977742448330684


In [251]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6137899954934656


In [252]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6817769718948323


**CASO 4:** 
**bathrooms, number_of_reviews, price**

In [253]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_reviews', 'D_price']]
Var_Dep = df['D_availability']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,

In [254]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[618 361]
 [507 733]]


In [255]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.670018281535649


In [256]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6088328075709779


In [257]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.5911290322580646


**CASO 5:** 
**accommodates, review_scores_rating, availability_30**

In [258]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_accommodates', 'D_reviews', 'D_availability']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,

In [259]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[703 417]
 [356 743]]


In [260]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6405172413793103


In [261]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6516448850833709


In [262]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6760691537761602


**CASO 6:** 
**instant_bookable, review_scores_rating, availability_90**

In [263]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['instant_bookabled', 'D_reviews', 'D_availability90']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,

In [ ]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[664 482]
 [425 648]]


In [265]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5734513274336284


In [266]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5912573231185219


In [267]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6039142590866728


**CASO 7:** 
**bathrooms, availability_30, host_listings**

In [274]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_availability', 'D_host_listings']]
Var_Dep = df['D_reviews']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,

In [275]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[659 447]
 [516 597]]


In [276]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5718390804597702


In [277]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.566020730058585


In [278]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.5363881401617251


**CASO 8:** 
**price, accommodates, review_scores_rating**

In [279]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_price', 'D_accommodates', 'D_rating']]
Var_Dep = df['D_reviews']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,

In [284]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[636 486]
 [452 645]]


In [281]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5702917771883289


In [282]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5772870662460567


In [285]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.5879671832269827


**CASO 9:** 
**bathrooms, review_scores_accuracy, review_scores_rating**

In [287]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_accuracy', 'D_rating']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,

In [288]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[536 592]
 [319 772]]


In [289]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5659824046920822


In [290]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5894547093285264


In [ ]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.7076076993583869


**CASO 10:** 
**availabilit_90, host_total_listings_count, number_of_reviews**

In [292]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_availability90', 'D_host_listings', 'D_reviews']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0,

In [293]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[548 581]
 [326 764]]


In [294]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5680297397769517


In [295]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5912573231185219


In [296]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.7009174311926606
